In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-83d86c47-d314-484f-a0ab-0c1910447c56;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 641ms :: artifacts dl 13

In [2]:
rawMetadata_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "10.128.0.12:9092") \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [3]:
from pyspark.sql import functions as sparkf

In [4]:
rawMetadata_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
onlyMetadata_df = rawMetadata_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import from_json, col

# Main schema
schema = StructType([
    StructField("locationId", StringType()),
    StructField("location", StringType()),
    StructField("parameter", StringType()),
    StructField("value", StringType()),
    StructField("date", StringType()),
    StructField("unit", StringType()),
    StructField("coordinates", StringType()),
    StructField("country", StringType()),
    StructField("city", StringType()),
    StructField("isMobile", StringType()),
    StructField("isAnalysis", StringType()),
    StructField("entity", StringType()),
    StructField("sensorType", StringType())
])

# Nested 'date' and 'coordinates' schema
date_schema = StructType([
    StructField("utc", StringType()),
    StructField("local", StringType())
])

coordinates_schema = StructType([
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType())
])

# Parse the JSON string column and convert it to a struct.
parsedData_df = onlyMetadata_df.withColumn("data", from_json("value", schema))

In [7]:
parsedData_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: string (nullable = true)
 |    |-- coordinates: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- isMobile: string (nullable = true)
 |    |-- isAnalysis: string (nullable = true)
 |    |-- entity: string (nullable = true)
 |    |-- sensorType: string (nullable = true)



In [8]:
# Select only the nested fields
unNested_df = parsedData_df.select("key", "data.*")

In [9]:
unNested_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [10]:
# Parse the JSON string column and convert it to a struct.
unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- date: struct (nullable = true)
 |    |-- utc: string (nullable = true)
 |    |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- locationId: string (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- parameter: string (nullable = true)
 |    |-- value: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- unit: st

In [11]:
extractedDateLatLong_df = unNested_df.withColumn("data", from_json("value", schema))\
.withColumn("date", from_json("date", date_schema))\
.withColumn("coordinates", from_json("coordinates", coordinates_schema))\
.select('key',
 'locationId',
 'location',
 'parameter',
 'value',
 'date.*',
 'unit',
 'coordinates.*',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType')

In [12]:
extractedDateLatLong_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)



In [13]:
final_df = extractedDateLatLong_df\
.withColumn('sourceUnixSTP',sparkf.unix_timestamp(sparkf.col('utc'), "yyyy-MM-dd'T'HH:mm:ssXXX"))

In [14]:
final_df.columns

['key',
 'locationId',
 'location',
 'parameter',
 'value',
 'utc',
 'local',
 'unit',
 'latitude',
 'longitude',
 'country',
 'city',
 'isMobile',
 'isAnalysis',
 'entity',
 'sensorType',
 'sourceUnixSTP']

In [15]:
final_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- locationId: string (nullable = true)
 |-- location: string (nullable = true)
 |-- parameter: string (nullable = true)
 |-- value: string (nullable = true)
 |-- utc: string (nullable = true)
 |-- local: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- isMobile: string (nullable = true)
 |-- isAnalysis: string (nullable = true)
 |-- entity: string (nullable = true)
 |-- sensorType: string (nullable = true)
 |-- sourceUnixSTP: long (nullable = true)



In [16]:
#agg_df = final_df.groupBy('value').agg(sparkf.max(sparkf.col('value')))

In [ ]:
from pyspark.sql import functions as F

def max_value(df, epoch_id):
    df = df.withColumn("value", df["value"].cast("float"))
    max_value_row = df.orderBy(df.value.desc()).first()
    if max_value_row is not None and max_value_row.value > 60:
        print(f"ALERT! Batch: {epoch_id}, Max value: {max_value_row.value} exceeded 60 at date: {max_value_row.local}!")
    elif max_value_row is not None:
        print(f"Batch: {epoch_id}, Max value: {max_value_row.value} at date: {max_value_row.local}")
    else:
        print(f"Batch: {epoch_id}, No data")

query = final_df.writeStream \
    .foreachBatch(max_value) \
    .outputMode("update") \
    .trigger(processingTime='5 seconds') \
    .start()

query.awaitTermination()


23/06/17 14:49:36 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fe8e477c-0dcc-40d1-adca-8ab2e2f19da3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


Batch: 0, No data


23/06/17 14:49:46 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6973 milliseconds


Batch: 1, Max value: 34.0 at date: 2022-01-28T10:00:00+07:00


23/06/17 14:49:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 6330 milliseconds


Batch: 2, Max value: 29.0 at date: 2022-01-29T17:00:00+07:00
Batch: 3, Max value: 24.0 at date: 2022-01-31T01:00:00+07:00
Batch: 4, Max value: 19.0 at date: 2022-02-02T01:00:00+07:00
Batch: 5, Max value: 20.0 at date: 2022-02-03T19:00:00+07:00
Batch: 6, Max value: 28.0 at date: 2022-02-05T09:00:00+07:00
Batch: 7, Max value: 42.0 at date: 2022-02-06T23:00:00+07:00
ALERT! Batch: 8, Max value: 63.0 exceeded 60 at date: 2022-02-07T22:00:00+07:00!
ALERT! Batch: 9, Max value: 63.0 exceeded 60 at date: 2022-02-08T16:00:00+07:00!
Batch: 10, Max value: 30.0 at date: 2022-02-10T12:00:00+07:00
Batch: 11, Max value: 31.0 at date: 2022-02-12T17:00:00+07:00
Batch: 12, Max value: 38.0 at date: 2022-02-13T03:00:00+07:00
Batch: 13, Max value: 29.0 at date: 2022-02-15T13:00:00+07:00
Batch: 14, Max value: 51.0 at date: 2022-02-17T00:00:00+07:00
ALERT! Batch: 15, Max value: 61.0 exceeded 60 at date: 2022-02-17T10:00:00+07:00!
Batch: 16, Max value: 25.0 at date: 2022-02-18T11:00:00+07:00
Batch: 17, Max val

In [ ]:
query.stop()